In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import multiprocessing
import pickle
import os
import psutil
import gc
from PF_Aso1_0_CPU import *

In [2]:
if __name__ == '__main__':

    workdir = os.path.dirname(os.getcwd())
    srcdir = os.getcwd()
    datadir = workdir + '/data/'
    outputdir = workdir + '/output/'

    seed = 7

    obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
    obs_series = np.array(obs_series.iloc[:,1:]).T

    T = obs_series.shape[1]
    N = 100000
    Λ_scale = 1.0
    cd_scale = 1.0

    case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
    try: 
        casedir = outputdir + case  + '/'
        os.mkdir(casedir)
    except:
        casedir = outputdir + case  + '/'
    print('1',psutil.Process().memory_full_info().rss/(1024*1024))
    np.random.seed(seed)
    start_time = time.time()

    D_0 = obs_series[:,[0]]

    Input_0 = [[D_0, Λ_scale, cd_scale] for i in range(N)]
    print('2',psutil.Process().memory_full_info().rss/(1024*1024))
    pool = multiprocessing.Pool()
    Output_0 = pool.map(init, tqdm(Input_0))
    print('3',psutil.Process().memory_full_info().rss/(1024*1024))
    del(Input_0)
    gc.collect()
    print('4',psutil.Process().memory_full_info().rss/(1024*1024))
    D_t_next = obs_series[:,[1]]
    Input = [[D_t_next, Output_0[i][1], Output_0[i][2], seed+i] for i in range(N)]
    print('5',psutil.Process().memory_full_info().rss/(1024*1024))
    del(Output_0)
    gc.collect()
    print('6',psutil.Process().memory_full_info().rss/(1024*1024))
    # with open(casedir + 'θ_0.pkl', 'wb') as f:
    #     pickle.dump(θ_t_particle, f)
    # with open(casedir + 'X_0.pkl', 'wb') as f:
    #     pickle.dump(X_t_particle, f)
    # with open(casedir + 'H_0.pkl', 'wb') as f:
    #     pickle.dump(H_t_particle, f)
    # with open(casedir + 'count_0.pkl', 'wb') as f:
    #     pickle.dump(list(np.ones(N)), f)
    # with open(casedir + 'w_0.pkl', 'wb') as f:
    #     pickle.dump(list(np.ones(N)/N), f)
    run_time = time.time() - start_time
    print(run_time)    
    for t in tqdm(range(T-1)):
        print('7',psutil.Process().memory_full_info().rss/(1024*1024))
        pool = multiprocessing.Pool()
        Output = pool.map(recursive, Input)
        print('8',psutil.Process().memory_full_info().rss/(1024*1024))
        del(Input)
        gc.collect()
        print('9',psutil.Process().memory_full_info().rss/(1024*1024))
        θ_t_next_particle = [i[0] for i in Output]
        print('10',psutil.Process().memory_full_info().rss/(1024*1024))
        with open(casedir + 'θ_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(θ_t_next_particle, f)
        del(θ_t_next_particle)
        gc.collect()
        print('11',psutil.Process().memory_full_info().rss/(1024*1024))
        ν_t_next_particle = [i[3] for i in Output]    

        # with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(X_t_next_particle, f)
        # with open(casedir + 'H_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(H_t_next_particle, f)

        w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)
        print('12',psutil.Process().memory_full_info().rss/(1024*1024))
        del(ν_t_next_particle)
        gc.collect()
        print('13',psutil.Process().memory_full_info().rss/(1024*1024))
        try:
            count_all = sp.stats.multinomial.rvs(N, w_t_next)
        except:
            for i in range(w_t_next.shape[0]):
                if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                    w_t_next[i] = w_t_next[i] - (np.sum(w_t_next[:-1]) - 1)
                    break
            count_all = sp.stats.multinomial.rvs(N, w_t_next)
        print('14',psutil.Process().memory_full_info().rss/(1024*1024))
        # with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(w_t_next, f)
        del(w_t_next)
        gc.collect()
        print('15',psutil.Process().memory_full_info().rss/(1024*1024))
        # with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(count_all, f)

        D_t_next = obs_series[:,[t+1]]
        Input = []
        for i in range(N):
            if count_all[i] != 0:
                for n in range(count_all[i]):
                    Input.append([D_t_next, Output[i][1], Output[i][2], seed+t+i])
        print('16',psutil.Process().memory_full_info().rss/(1024*1024))
        del(Output)
        print('17',psutil.Process().memory_full_info().rss/(1024*1024))
        gc.collect()
        print('18',psutil.Process().memory_full_info().rss/(1024*1024))
        del(count_all) 
        print('19',psutil.Process().memory_full_info().rss/(1024*1024))
        gc.collect()
        print('20',psutil.Process().memory_full_info().rss/(1024*1024))

1 165.703125
2 177.62890625


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [01:06<00:00, 1494.73it/s]


3 568.86328125
4 568.98828125
5 569.234375
6 650.6796875
89.12994480133057


  0%|                                                                                                                                                                             | 0/282 [00:00<?, ?it/s]

7 650.76171875
8 919.33984375
9 999.63671875
10 999.734375
11 1365.828125
12 1366.33203125
13 1366.2265625
14 1366.703125
15 1366.703125
16 1366.94921875


...

17 1421.09765625
18 1421.09765625
19 1421.09765625
20 1421.1015625
7 1421.13671875
8 896.203125
9 958.8359375
10 958.98046875
11 1298.39453125
12 1298.62109375
13 1298.62109375
14 1298.6875
15 1298.19921875
16 1298.20703125
17 1313.54296875
18 1313.54296875
19 1313.54296875


  1%|█▏                                                                                                                                                                 | 2/282 [01:46<4:06:38, 52.85s/it]

20 1313.54296875
7 1313.578125


  1%|█▏                                                                                                                                                                 | 2/282 [02:06<4:54:43, 63.15s/it]Process SpawnPoolWorker-31:
Process SpawnPoolWorker-32:
Process SpawnPoolWorker-29:
Process SpawnPoolWorker-27:

Traceback (most recent call last):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/45th Version, PF memory save/src/PF_Aso1_0_CPU.py"

KeyboardInterrupt: 